In [1]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import scala.util.{Failure, Success, Try}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import com.mongodb.spark._
import com.mongodb.spark.config._
import org.bson._

import java.time.LocalDateTime
import java.time.LocalDate
import java.time.format.DateTimeFormatter

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
47,application_1707107681556_0048,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._
import scala.util.{Failure, Success, Try}
import org.apache.spark.sql.types._
import org.apache.spark.sql.SparkSession
import com.mongodb.spark._
import com.mongodb.spark.config._
import org.bson._
import java.time.LocalDateTime
import java.time.LocalDate
import java.time.format.DateTimeFormatter


In [ ]:
val spark = (
    SparkSession
    .builder()
    .appName("Frequency")
    .config("spark.cores.max", 6)
    .config("spark.executor.cores", 6)
    .config("spark.executor.memory", "36g")
    .getOrCreate()
    )

In [ ]:
def dateTimeFormatter = DateTimeFormatter.ofPattern("yyyyMMddHHmmss")
def appName: String = LocalDateTime.now().format(dateTimeFormatter)
def previousDay(minusDays: Long) = {
    LocalDateTime.now().minusDays(minusDays).format(dateTimeFormatter).substring(0, 8)
}

In [ ]:
val mongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
val matchPipe = String.format("{ $match: { updateDate: { $gte: '%s' } } }", previousDay(120)) # %s는 정규식공백이 아니라, previousDay의 결과가 대체되는 것
val df = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "Price")
    .option("aggregation.pipeline", matchPipe)
    .load()
    .drop("_id")
    )

In [ ]:
val nameFilter = udf((x: String) => {
    val pattern = "(금융|보험|기업인수목적|증권)".r
    pattern.findFirstIn(x) match {
      case Some(s) => true
      case None => false
    }
})


val stockInfoDf = (
    spark.read.format("mongodb")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "StockInformation")
    .load()
    .drop("_id")
    .where(col("stockType") === "보통주")
    .where(col("stockClassify") !== "부동산투자회사")
    .where(col("stockClassify") !== "주식예탁증권")
    .withColumn("regex", nameFilter(col("stockFullName"))) 
    .where(col("regex") === false)
    .select("stockCode", "stockType")
    )

In [ ]:
val mergeDf = df.join(stockInfoDf, Seq("stockCode"), "left")

Event = (1번 상황 | 2번 상황) 이면 1
Price(t)- price(t-1) <= price(t-1) * (-0.1)  즉, 직전 거래일 대비 주가가 10%이상 하락한 경우, 
직전 거래일과의 차이가 5일 연속으로 음수인 경우, 즉, 5거래일 동안 연속 하락한 경우

In [ ]:
val window = Window.partitionBy(col("stockCode")).orderBy(col("updateDate"))
// val rankWindow = Window.partitionBy(col("stockCode")).orderBy(col("updateDate").desc)
val hf = (
    mergeDf
    .where(col("stockType").isNotNull)
    .where(col("closingPrice").isNotNull)
    .withColumn("market", when(col("classify") === "KOSPI", 1).otherwise(0))
    .select("stockCode", "stockFullName", "market", "closingPrice", "prepare", "openPrice", "highPrice", "lowPrice", "tradingVolume", "transactionAmount", "updateDate")
    .withColumn("closingPricePrev", lag(col("closingPrice"), 1).over(window))
    .withColumn("closingPriceNext", lag(col("closingPrice"), -1).over(window))
    .withColumn("closingPriceDiff", col("closingPrice") - col("closingPricePrev"))
    .withColumn("closingPriceReturn", log(col("closingPrice") / col("closingPricePrev")))
    .withColumn("closingPriceEvt", when(col("closingPriceDiff") <= (col("closingPricePrev") * -0.1), 1).otherwise(0))
    .withColumn("closingPrice5Min", min(col("closingPriceDiff")).over(window.rowsBetween(-4, 0)))
    .withColumn("closingPrice5Max", max(col("closingPriceDiff")).over(window.rowsBetween(-4, 0)))
    .withColumn("closingPrice5", when((col("closingPrice5Max").cast("long") < 0) && (col("closingPrice5Min").cast(LongType) < 0), 1).otherwise(0))
    .withColumn("tradingVolumeLog", log(col("tradingVolume")))
    .withColumn("closingPriceReturn5days", mean(col("closingPriceReturn")).over(window.rowsBetween(-4, 0)))
    .withColumn("closingPriceReturn22days", mean(col("closingPriceReturn")).over(window.rowsBetween(-21, 0)))
    .withColumn("tradingVolumeVolatility5days", (variance(col("tradingVolume")).over(window.rowsBetween(-4, 0))))
    .withColumn("tradingVolumeVolatility22days", (variance(col("tradingVolume")).over(window.rowsBetween(-21, 0))))
    .withColumn("tradingVolumeVolatility5daysLog", log(col("tradingVolume")))
    .withColumn("tradingVolumeVolatility22daysLog", log(col("tradingVolume")))
    .withColumn("event", when((col("closingPriceEvt") === 1) || (col("closingPrice5") === 1), 1).otherwise(0))
    .select("stockCode", "stockFullName", "market", "updateDate", "closingPrice", "tradingVolume", "transactionAmount", "closingPricePrev", "closingPriceNext", "closingPriceDiff", "closingPriceReturn", "closingPrice5Min", "closingPrice5Max", "closingPrice5", "tradingVolumeLog", "closingPriceReturn5days", "closingPriceReturn22days", "tradingVolumeVolatility5days", "tradingVolumeVolatility22days", "tradingVolumeVolatility5daysLog", "tradingVolumeVolatility22daysLog", "event")
//     .withColumn("rank", row_number().over(rankWindow))
    )

In [ ]:
"""
(
    hf
    .where(col("updateDate") > "20230630")
    .na.fill(0)
    .write.format("mongodb")
    .mode("append")
    .option("upsertDocument", "true")
    .option("idFieldList", "updateDate,stockCode")
    .option("spark.mongodb.read.connection.uri", mongoUrl)
    .option("spark.mongodb.write.connection.uri", mongoUrl)
    .option("database", "coreEngine")
    .option("collection", "PriceFeatures")
    .save()
    )
"""

In [ ]:
spark.stop()